เริ่มจากการโหลดโมเดลที่ต้องการเข้ามา พร้อมทดลองใช้งาน

In [ ]:
from langchain.llms import GPT4All

# Load the model
llm = GPT4All(model='./mistral-7b-openorca.Q4_0.gguf')

# Define a sentence to complete
sentence = "The capital of Thailand is "

# Complete the sentence
response = llm.generate([sentence])
print(response.generations[0][0].text)

ในการสร้างผู้ช่วยส่วนตัว เราต้องเชื่อมต่อdataเข้ากับ LLM LlamaIndex ช่วยเราสำเร็จส่วนนี้ได้ด้วยโค้ดไม่กี่บรรทัด แต่ก่อนอื่น มาเตรียมembedding model และ sentence splitter เพื่อใช้ในกระบวนการเตรียมdataให้อยู่ในรูปแบบที่ง่ายต่อการใช้งานLLM

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core.indices.prompt_helper import PromptHelper
from llama_index.core.node_parser import SentenceSplitter

# An embedding model used to structure text into representations
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

# PromptHelper can help deal with LLM context window and token limitations
prompt_helper = PromptHelper(context_window=2048)

# SentenceSplitter used to split our data into multiple chunks
# Only a number of relevant chunks will be retrieved and fed into LLMs
node_parser = SentenceSplitter(chunk_size=300, chunk_overlap=20)

เราสามารถตั้ง global setting เพื่อใช้ส่วนประกอบทั้งหมดอย่าง llm, embed_model, prompt_helper, node_parser ดังโค้ดด้านล่างซึ่งมีประโยชน์ดังนี้:

1.การตั้งค่าระดับ global: เราไม่ต้องกำหนดค่าเหล่านี้ในแต่ละที่ที่ใช้ LlamaIndex หรือส่วนประกอบต่างๆ ซึ่งทำให้โค้ดของเราดูสะอาดตาและจัดการได้ง่ายขึ้น
2.ความยืดหยุ่น: เราสามารถปรับเปลี่ยนการตั้งค่าของระบบได้ในที่เดียว โดยไม่ต้องไปเปลี่ยนโค้ดที่ใช้แต่ละฟังก์ชันที่เกี่ยวข้อง
3.การใช้ร่วมกันของหลายคอมโพเนนต์: เมื่อเรากำหนดค่าเหล่านี้ใน Settings คอมโพเนนต์ต่างๆ ที่ใช้ LlamaIndex เช่น VectorStoreIndex หรือ QueryEngine จะสามารถใช้การตั้งค่าเดียวกันได้โดยอัตโนมัติ


In [ ]:
from llama_index.core import Settings

# Configure a global setting for our app so that the VectorStoreIndex and the QueryEngine can use these components by default.
Settings.llm = llm
Settings.embed_model = embed_model
Settings.prompt_helper = prompt_helper
Settings.node_parser = node_parser

ต่อไปเราจะโหลดdataเข้ามายังdocument ใช้service container เพื่อprocess document สร้างindexและเปลี่ยนindexให้เป็น query engine เพื่อที่เราจะใช้มันในการถามคำถาม

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

# Load data.txt into a document
document = SimpleDirectoryReader(input_files=['./data.txt']).load_data()

# Process data (chunking, embedding, indexing) and store them
index = VectorStoreIndex.from_documents(document)

# Build a query engine from the index
query_engine = index.as_query_engine()

query engineนี้คือผู้ช่วยของเรา ซึ่งสามารถทำงานง่ายๆกับdataของเราได้ ตัวอย่างเช่น:

In [ ]:
response = query_engine.query('Give me my calendar.')
print(response)

สำหรับการทำงานสรุปข้อความ ควรสร้างindexใหม่ที่เหมาะกับงานนี้ดีกว่า โดยใช้SummaryIndexซึ่งจะสรุปคำตอบจากทุกchunksของdataเปรียบเทียบกับ top-k chunksที่อยู่ใน VectorStoreIndex

In [ ]:
from llama_index.core import SummaryIndex

# Load data.txt into a document
document = SimpleDirectoryReader(input_files=['./data.txt']).load_data()

# Process data (chunking, embedding, indexing) and store them
summary_index = SummaryIndex.from_documents(document)

# Build a summary engine from the index
summary_engine = summary_index.as_query_engine(response_mode="tree_summarize")

ทดลองสรุปเนื้อหาข่าวดู

In [ ]:
response = summary_engine.query("What is a summary of this collection of text?")
print(response)